# Clean Eurorack Dataset

<br>

Remove any modules by their function, such as 'Blank Panel' or 'Expander'. Drop duplicate modules with same name and manufacturer. Uses a pretty slow and clunky way of detecting and removing duplicates with a panel descriptor, e.g. Make Noise - MATHS (black panel) will get dropped by the original MATHS. Removed modules are merged with the original to try and obtain more accurate data. 

<br>

Nan values can be delt with in this script.

In [1]:
import numpy as np
import pandas as pd

#string matching
from thefuzz import fuzz

In [2]:
#load dataset
df = pd.read_csv('1500_most_popular_eurorack_modules_2024_UNCLEANED.csv', index_col=0)
df.head()

,Name,Manufacturer,Functions,HP,Depth,+12V (mA),-12V (mA),+5V (mA),Price (€),Racks,Rating,Votes,Available,Approved
0,Plaits,Mutable instruments,"['Noise', 'Oscillator', 'Synth Voice', 'Drum',...",12,25.0,50.0,5.0,0.0,250.0,62584,4.57,352.0,Discontinued,0
1,Pamela's NEW Workout,ALM Busy Circuits,"['Clock Generator', 'Digital', 'LFO', 'Clock M...",8,22.0,50.0,1.0,0.0,279.0,55808,4.75,382.0,Discontinued,0
2,Morphagene,Make Noise,"['Sampling', 'Delay', 'Digital', 'Dual/Stereo'...",20,30.0,165.0,20.0,0.0,569.0,49149,4.63,217.0,Available,1
3,Maths,Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,268.0,49099,4.75,562.0,NaN,1
4,Disting mk4,Expert Sleepers,"['Effect', 'Quantizer', 'Utility', 'LFO', 'Osc...",4,42.0,51.0,19.0,0.0,189.0,48139,4.48,265.0,Available,0


In [3]:
#the Make Noise - Maths module is a good example that contains multiple variants
df[df.Name.str.contains('Maths', case=False)]

,Name,Manufacturer,Functions,HP,Depth,+12V (mA),-12V (mA),+5V (mA),Price (€),Racks,Rating,Votes,Available,Approved
3,Maths,Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,268.0,49099,4.75,562.0,NaN,1
5,Maths (white knobs),Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,319.0,44616,4.69,176.0,Available,0
38,MATHS (black panel),Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,325.0,17400,4.50,42.0,Available,0
280,MATHS,Make Noise,['Function Generator'],20,24.0,60.0,60.0,0.0,325.0,5771,4.60,98.0,Discontinued,0
311,Maths,Make Noise,"['Envelope Generator', 'Envelope Follower', 'L...",20,24.0,60.0,50.0,0.0,245.0,5335,4.49,83.0,Discontinued,0
759,Maths v2 (Grayscale panel),Grayscale,"['Clock Generator', 'CV Modulation', 'Envelope...",20,24.0,60.0,50.0,NaN,261.0,2517,4.60,15.0,NaN,1
838,20hp Blank Panel (Maths Lightning Bolts),Make Noise,['Blank Panel'],20,NaN,NaN,NaN,NaN,14.0,2276,4.25,8.0,NaN,1
1116,Maths v2 (Grayscale black panel),Grayscale,"['Clock Generator', 'CV Modulation', 'Envelope...",20,24.0,60.0,50.0,0.0,261.0,1707,4.85,13.0,Available,1


In [4]:
#Data Cleaning
#---------------------------------------------------------

#drop based on fucntion
#remove blank panels, power modules, and expander modules

#COMMENT OUT IF YOU WANT TO HAVE THESE IN THE DATA SET
df = df[~df.Functions.str.contains('Blank Panel')]
# df = df[~df.Functions.str.contains('Power')]
# df = df[~df.Functions.str.contains('Expander')]


#remove duplicates that share the same name and manufacturer (case insensitive)
#create to column to contain case insensitive name and manufacturer
df['Name Lower'] = df['Name'].str.lower()
df['Manufacturer Lower'] = df['Manufacturer'].str.lower()

#merge modules with same name and manufacturer
df_sum = df.groupby(['Name Lower', 'Manufacturer Lower'], as_index=False).agg({'Racks': 'sum', 'Votes': 'sum', 'Rating': 'mean'})
#sum racks and votes, find mean of rating


#drop duplicates on original df
df = df.drop_duplicates(subset=['Name Lower', 'Manufacturer Lower'], keep='first')

#merg df_sum and df
df = df.merge(df_sum, on=['Name Lower', 'Manufacturer Lower'], how='left')
#replace old info (Old_x) with new (New_y)
df['Racks_x'] = df['Racks_y']
df['Votes_x'] = df['Votes_y']
df['Rating_x'] = round(df['Rating_y'], 2)
#remove unwanted columns
df = df.drop(columns=['Name Lower', 'Manufacturer Lower', 'Racks_y', 'Votes_y', 'Rating_y'])
df = df.rename(columns={'Racks_x': 'Racks', 'Votes_x': 'Votes', 'Rating_x': 'Rating'})

#resort by Racks
df = df.sort_values(by='Racks', ascending=False)
df = df.reset_index(drop=True)

print('Dataset Length:', len(df))

Dataset Length: 1458


In [5]:
#duplicates persist with different panels
df[df.Name.str.contains('Maths', case=False)]

,Name,Manufacturer,Functions,HP,Depth,+12V (mA),-12V (mA),+5V (mA),Price (€),Racks,Rating,Votes,Available,Approved
1,Maths,Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,268.0,60205,4.61,743.0,NaN,1
5,Maths (white knobs),Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,319.0,44616,4.69,176.0,Available,0
38,MATHS (black panel),Make Noise,"['Attenuator', 'CV Modulation', 'Envelope Foll...",20,24.0,60.0,50.0,0.0,325.0,17400,4.50,42.0,Available,0
742,Maths v2 (Grayscale panel),Grayscale,"['Clock Generator', 'CV Modulation', 'Envelope...",20,24.0,60.0,50.0,NaN,261.0,2517,4.60,15.0,NaN,1
1089,Maths v2 (Grayscale black panel),Grayscale,"['Clock Generator', 'CV Modulation', 'Envelope...",20,24.0,60.0,50.0,0.0,261.0,1707,4.85,13.0,Available,1


In [ ]:
#remove duplicates with panel descriptor

#create new column to store string similarity
df.insert(14, 'Similarity', np.zeros(len(df)))

#sort df by length of name
#original module will have shorter name
df = df.sort_values(by="Name", key=lambda x: x.str.len())
df = df.reset_index(drop=True)

for target_module in range(0, len(df)):

    #slice from target module to end
    dupes = df[target_module:]
    
    #test all modules and find modules with similar names to target
    for search_index in range(0, len(dupes)):
        
        similarity = fuzz.token_set_ratio(df.iloc[target_module, 0], dupes.iloc[search_index, 0])
    
        #disrigard if target and test modules do not share the same manufacturer	
        if df.iloc[target_module, 1] != dupes.iloc[search_index, 1]:
            similarity = 0
    
        dupes.iloc[search_index, 14] = similarity
        
    #drop all duplicate modules that have a similarity score less than threshold
    dupes = dupes[~(dupes['Similarity'] < 95 )]

    #check if dupe df contains more than one row(the target itself)
    if len(dupes) > 1:
        
        #modules that are similar to target and contain panel descriptor
        panel_dupes = dupes[dupes.Name.str.contains('panel', case=False)|dupes.Name.str.contains('black', case=False)
                            |dupes.Name.str.contains('white', case=False)|dupes.Name.str.contains('silver', case=False)
                            |dupes.Name.str.contains('gold', case=False)|dupes.Name.str.contains('edition', case=False)
                            |dupes.Name.str.contains('grey', case=False)]
        
    
        #count how many modules have been dropped
        #used when dropping from df
        modules_dropped = 0
        
        panel_rating = 0
        #loop through all dupe modules containing panel descriptors
        for search_index in range(0, len(panel_dupes)):
            
            #check if target and test module are the same by comparing index
            if panel_dupes.axes[0][search_index] != target_module:
                
                #assume if HP match, modules are the same and can be removed
                if panel_dupes.iloc[search_index, 3] == df.iloc[target_module, 3]:

                    #sum racks to target
                    df.iloc[target_module, 9] += panel_dupes.iloc[search_index, 9]
                    
                    #sum votes 
                    df.iloc[target_module, 11] += panel_dupes.iloc[search_index, 11]
                    
                    #sum panel rating
                    panel_rating += panel_dupes.iloc[search_index, 10]
                    
                    #drop panel dupes from dupes
                    dupes = dupes.drop(panel_dupes.axes[0][search_index])

                    #display which dupe modules have been removed, and by which target module
                    print(df.iloc[panel_dupes.axes[0][search_index]-modules_dropped, 0], 'Dropped by', df.iloc[target_module, 0])
                
                    #drop panel dupe from df
                    df = df.drop(panel_dupes.axes[0][search_index]-modules_dropped)
    
                    #correct index when panel dupes have been dropped for df
                    modules_dropped += 1
                    
                    df = df.reset_index(drop=True)

        df.iloc[target_module, 10] = round((df.iloc[target_module, 10] + panel_rating) / (modules_dropped + 1), 2)

In [ ]:
#drop similarity column and reorder by Racks (popularity)
df = df.drop(['Similarity'], axis=1)
df = df.sort_values(by='Racks', ascending=False)
df = df.reset_index(drop=True)

In [ ]:
#trim dataset down to L

#L = 1000
#df = df[:L]

In [ ]:
len(df)

In [6]:
#removing nan values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          1458 non-null   object 
 1   Manufacturer  1458 non-null   object 
 2   Functions     1458 non-null   object 
 3   HP            1458 non-null   int64  
 4   Depth         1390 non-null   float64
 5   +12V (mA)     1390 non-null   float64
 6   -12V (mA)     1378 non-null   float64
 7   +5V (mA)      1151 non-null   float64
 8   Price (€)     1444 non-null   float64
 9   Racks         1458 non-null   int64  
 10  Rating        1455 non-null   float64
 11  Votes         1458 non-null   float64
 12  Available     1347 non-null   object 
 13  Approved      1458 non-null   int64  
dtypes: float64(7), int64(3), object(4)
memory usage: 159.6+ KB


In [ ]:
#fill na values

#find means to fill certain na values
#round
mean_depth = round(df['Depth'].mean(), 1)
mean_pos12 = round(df['+12V (mA)'].mean(), 1)
mean_neg12 = round(df['-12V (mA)'].mean(), 1)
mean_rating = round(df['Rating'].mean(), 2)
mean_price = round(df['Price (€)'].mean(), 1)

#create dictionary to fill na values
to_fill = {'Depth': mean_depth,
           '+12V (mA)': mean_pos12,
           '-12V (mA)': mean_neg12,
           '+5V (mA)': 0,
           'Price (€)': mean_price,
           'Rating': mean_rating,
           'Available': 'No Info'
          } 

df = df.fillna(value=to_fill)

In [ ]:
df.info()

In [ ]:
#save clearned dataset ready for analysis
#N = 1000
YEAR = 2024
df.to_csv(f"{N}_most_popular_eurorack_modules_{YEAR}.csv") 

## Wes Leggo-Morrell 2024

#### **[Instagram](https://www.instagram.com/modular.mooch)**

#### **[GitHub](https://github.com/WesDaMooch)**